<a href="https://colab.research.google.com/github/IosuAV/ibm_practice/blob/main/Interactive_Visual_Analytics_with_Folium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.

Let's first import required Python packages for this lab:


In [ ]:
!pip install folium

In [ ]:
import folium
import pandas as pd

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


In [ ]:
## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [ ]:
# Download and read the `spacex_launch_geo.csv`
URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df=pd.read_csv(URL)

Now, you can take a look at what are the coordinates for each site.


In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [ ]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [ ]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [ ]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label

# Datos de los centros de lanzamiento
launch_sites = [
    {'name': 'CCAFS LC-40', 'lat': 28.562302, 'long': -80.577356},
    {'name': 'CCAFS SLC-40', 'lat': 28.563197, 'long': -80.576820},
    {'name': 'KSC LC-39A', 'lat': 28.573255, 'long': -80.646895},
    {'name': 'VAFB SLC-4E', 'lat': 34.632834, 'long': -120.610745},
]

# Agrega los círculos y marcadores para cada centro de lanzamiento
for site in launch_sites:
    coordinate = [site['lat'], site['long']]
    circle = folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(site['name']))
    marker = folium.map.Marker(
        coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site['name'],
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

# Muestra el mapa
site_map

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.

In [ ]:
# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not

In [ ]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [ ]:
marker_cluster = MarkerCluster()

*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [ ]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # Crea un marcador para cada lanzamiento
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],  # Coordenadas del lanzamiento
        icon=folium.Icon(color='white', icon_color=record['marker_color']),  # Color del marcador
        popup=record['Launch Site'],  # Mensaje emergente con el nombre del sitio de lanzamiento
    )
    # Agrega el marcador al cluster
    marker_cluster.add_child(marker)

site_map

In [ ]:
# Obtén el número de lanzamientos en el DataFrame
num_lanzamientos_df = len(spacex_df)

# Obtén el número de marcadores en el cluster
# Obtiene las coordenadas de los marcadores
bounds = marker_cluster.get_bounds()
# Si hay marcadores, el número de marcadores es igual al número de coordenadas
num_marcadores_cluster = len(bounds) if bounds else 0

# Imprime los resultados
print("Número de lanzamientos en el DataFrame:", num_lanzamientos_df)
print("Número de marcadores en el cluster:", num_marcadores_cluster)

# Compara los valores
if num_lanzamientos_df == num_marcadores_cluster:
    print("El número de lanzamientos y marcadores coincide.")
else:
    print("El número de lanzamientos y marcadores no coincide.")

Número de lanzamientos en el DataFrame: 56
Número de marcadores en el cluster: 2
El número de lanzamientos y marcadores no coincide.


In [ ]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # Crea un marcador para cada lanzamiento
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],  # Coordenadas del lanzamiento
        icon=folium.Icon(color='white', icon_color=record['marker_color']),  # Color del marcador
        popup=record['Launch Site'],  # Mensaje emergente con el nombre del sitio de lanzamiento
    )
    # Agrega el marcador al cluster
    marker_cluster.add_child(marker)

site_map

# Obtén el número de lanzamientos en el DataFrame
num_lanzamientos_df = len(spacex_df)

# Obtén el número de marcadores en el cluster (indirectamente)
num_marcadores_cluster = len(marker_cluster._children) # Accede al diccionario _children y cuenta sus elementos

# Imprime los resultados
print("Número de lanzamientos en el DataFrame:", num_lanzamientos_df)
print("Número de marcadores en el cluster:", num_marcadores_cluster)

# Compara los valores
if num_lanzamientos_df == num_marcadores_cluster:
    print("El número de lanzamientos y marcadores coincide.")
else:
    print("El número de lanzamientos y marcadores no coincide.")


Número de lanzamientos en el DataFrame: 56
Número de marcadores en el cluster: 169
El número de lanzamientos y marcadores no coincide.


In [ ]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed,
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # Crea un marcador para cada lanzamiento
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],  # Coordenadas del lanzamiento
        icon=folium.Icon(color='white', icon_color=record['marker_color']),  # Color del marcador
        popup=record['Launch Site'],  # Mensaje emergente con el nombre del sitio de lanzamiento
    )
    # Agrega el marcador al cluster
    marker_cluster.add_child(marker)

# Obtén el número de lanzamientos en el DataFrame
num_lanzamientos_df = len(spacex_df)

# Obtén el número de marcadores en el cluster (indirectamente)
num_marcadores_cluster = 0
for child in marker_cluster._children.values():
    if isinstance(child, folium.Marker):
        num_marcadores_cluster += 1

# Imprime los resultados
print("Número de lanzamientos en el DataFrame:", num_lanzamientos_df)
print("Número de marcadores en el cluster:", num_marcadores_cluster)

# Compara los valores
if num_lanzamientos_df == num_marcadores_cluster:
    print("El número de lanzamientos y marcadores coincide.")
else:
    print("El número de lanzamientos y marcadores no coincide.")

site_map  # Muestra el mapa al final

Número de lanzamientos en el DataFrame: 56
Número de marcadores en el cluster: 56
El número de lanzamientos y marcadores coincide.


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [ ]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.




In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Objetivo:

El objetivo de este código es definir una función llamada calculate_distance que calcula la distancia entre dos puntos geográficos dados sus coordenadas de latitud y longitud. La función utiliza la fórmula de Haversine para realizar el cálculo.

Código explicado:

from math import sin, cos, sqrt, atan2, radians:

Esta línea importa las funciones matemáticas necesarias para el cálculo de la distancia: sin, cos, sqrt, atan2 y radians. Estas funciones se utilizan en la fórmula de Haversine.
def calculate_distance(lat1, lon1, lat2, lon2)::

Esta línea define la función calculate_distance que toma cuatro argumentos:
lat1: Latitud del primer punto.
lon1: Longitud del primer punto.
lat2: Latitud del segundo punto.
lon2: Longitud del segundo punto.
# approximate radius of earth in km R = 6373.0:

Estas líneas definen una variable R que representa el radio aproximado de la Tierra en kilómetros. Este valor se utiliza en la fórmula de Haversine.
lat1 = radians(lat1) lon1 = radians(lon1) lat2 = radians(lat2) lon2 = radians(lon2):

Estas líneas convierten las coordenadas de latitud y longitud de grados a radianes utilizando la función radians. La fórmula de Haversine requiere que las coordenadas estén en radianes.
dlon = lon2 - lon1 dlat = lat2 - lat1:

Estas líneas calculan la diferencia de longitud (dlon) y la diferencia de latitud (dlat) entre los dos puntos.
`a = sin(dlat / 2)2 + cos(lat1) * cos(lat2) * sin(dlon / 2)2`:

Esta línea calcula una parte de la fórmula de Haversine, que representa la "distancia angular" entre los dos puntos en la superficie de la esfera.
c = 2 * atan2(sqrt(a), sqrt(1 - a)):

Esta línea calcula la distancia angular final c utilizando la función atan2 y la raíz cuadrada (sqrt).
distance = R * c:

Esta línea calcula la distancia real entre los dos puntos multiplicando la distancia angular c por el radio de la Tierra R.
return distance:

Esta línea devuelve la distancia calculada.
En resumen, esta función calculate_distance implementa la fórmula de Haversine para calcular la distancia entre dos puntos geográficos dados sus coordenadas de latitud y longitud. Puedes utilizar esta función para calcular las distancias entre los sitios de lanzamiento y los puntos de interés que has identificado en el mapa.

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.




In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline = calculate_distance(34.632834, -120.610745, 34.63564, -120.62511)

In [ ]:
# ... (código anterior del mapa y la función calculate_distance) ...

# Distancia a la costa (usando tus coordenadas)
distance_coastline = calculate_distance(34.632834, -120.610745, 34.63564, -120.62511)

# Coordenadas del punto en la costa
coastline_coordinates = [34.63564, -120.62511]

# Crea el marcador con la distancia
distance_marker = folium.Marker(
    coastline_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
    )
)

# Agrega el marcador al mapa
site_map.add_child(distance_marker)

# Muestra el mapa
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate


# Coordenadas del sitio de lanzamiento (VAFB SLC-4E en este caso)
launch_site_coordinates = [34.632834, -120.610745]

# Coordenadas del punto en la costa (las que usaste antes)
coastline_coordinates = [34.63564, -120.62511]

# Crea la PolyLine
lines = folium.PolyLine(locations=[launch_site_coordinates, coastline_coordinates], weight=1)

# Agrega la PolyLine al mapa
site_map.add_child(lines)

# Muestra el mapa
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>

*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>

A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>

In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site

In [ ]:
# find coordinate of the closet Los Angeles International Airport (The Theme Building)
# Cordinates: Lat: 33.94411  Lon: -120.42247
# distance_airport_LA = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_airport_LA = calculate_distance(34.632834, -120.610745, 33.94411, -118.40247)

In [ ]:
# Create and add a folium.Marker on your selected closest airport point on the map
# Display the distance between airport_LA point and launch site using the icon property
# for example
# Coordenadas del punto en la costa
airport_LA_coordinates = [33.94411, -118.40247]

# Crea el marcador con la distancia
distance_marker = folium.Marker(
    airport_LA_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_airport_LA),
    )
)

# Agrega el marcador al mapa
site_map.add_child(distance_marker)

# Muestra el mapa
site_map

In [ ]:
# Create a `folium.PolyLine` object using the airport coordinates and launch site coordinate
# Coordenadas del sitio de lanzamiento (VAFB SLC-4E en este caso)
launch_site_coordinates = [34.632834, -120.610745]

# Coordenadas del aeropuerto LA (las que usaste antes)
airport_LA_coordinates = [33.94411, -118.40247]

# Crea la PolyLine
lines = folium.PolyLine(locations=[launch_site_coordinates, airport_LA_coordinates], weight=1)

# Agrega la PolyLine al mapa
site_map.add_child(lines)

# Muestra el mapa
site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.
